In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
rental = pd.read_csv('resources/data/Apartment-List-Rent-Data-State_2018-6.csv')
rental.head()

,Location,Location_Type,Bedroom_Size,Price_2014_01,Price_2014_02,Price_2014_03,Price_2014_04,Price_2014_05,Price_2014_06,Price_2014_07,...,Price_2017_09,Price_2017_10,Price_2017_11,Price_2017_12,Price_2018_01,Price_2018_02,Price_2018_03,Price_2018_04,Price_2018_05,Price_2018_06
0,Alabama,State,Studio,573,573,574,574,575,576,578.0,...,602,602,602,601,600,597,598,600,604,608
1,Alabama,State,1br,624,624,625,625,626,627,629.0,...,656,655,655,655,653,650,650,653,658,662
2,Alabama,State,2br,758,758,759,759,760,762,765.0,...,797,795,796,795,794,790,790,793,799,804
3,Alabama,State,3br,1013,1013,1014,1015,1016,1018,1022.0,...,1065,1063,1063,1063,1061,1055,1056,1060,1068,1074
4,Alabama,State,4br,1178,1178,1179,1180,1182,1184,1188.0,...,1238,1236,1237,1236,1234,1228,1228,1233,1242,1250


## Data Wrangling

### Assess 

In [3]:
rental.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255 entries, 0 to 254
Data columns (total 57 columns):
Location         255 non-null object
Location_Type    255 non-null object
Bedroom_Size     255 non-null object
Price_2014_01    255 non-null int64
Price_2014_02    255 non-null int64
Price_2014_03    255 non-null int64
Price_2014_04    255 non-null int64
Price_2014_05    255 non-null int64
Price_2014_06    255 non-null int64
Price_2014_07    250 non-null float64
Price_2014_08    250 non-null float64
Price_2014_09    250 non-null float64
Price_2014_10    250 non-null float64
Price_2014_11    250 non-null float64
Price_2014_12    250 non-null float64
Price_2015_01    250 non-null float64
Price_2015_02    250 non-null float64
Price_2015_03    250 non-null float64
Price_2015_04    250 non-null float64
Price_2015_05    250 non-null float64
Price_2015_06    250 non-null float64
Price_2015_07    250 non-null float64
Price_2015_08    255 non-null int64
Price_2015_09    255 non-null int64
Pr

#### Findings
 - Location_type is a redundant column
 - Year and month are wide data instead of long
 - Only data to the end of 2017 is needed
 - Data is unavailable for pre-2014 (only four year)

### Clean

In [4]:
rental_copy = rental.copy()

Drop redundant columns 

In [5]:
rental_copy.drop(['Location_Type', 
                  'Price_2018_01', 
                  'Price_2018_02', 
                  'Price_2018_03', 
                  'Price_2018_04', 
                  'Price_2018_05', 
                  'Price_2018_06'], 
                 axis=1, inplace=True)
rental_copy.head()

,Location,Bedroom_Size,Price_2014_01,Price_2014_02,Price_2014_03,Price_2014_04,Price_2014_05,Price_2014_06,Price_2014_07,Price_2014_08,...,Price_2017_03,Price_2017_04,Price_2017_05,Price_2017_06,Price_2017_07,Price_2017_08,Price_2017_09,Price_2017_10,Price_2017_11,Price_2017_12
0,Alabama,Studio,573,573,574,574,575,576,578.0,580.0,...,595,597,599,601,603,603,602,602,602,601
1,Alabama,1br,624,624,625,625,626,627,629.0,631.0,...,647,650,652,655,656,656,656,655,655,655
2,Alabama,2br,758,758,759,759,760,762,765.0,766.0,...,787,789,792,795,797,797,797,795,796,795
3,Alabama,3br,1013,1013,1014,1015,1016,1018,1022.0,1024.0,...,1051,1055,1058,1063,1065,1065,1065,1063,1063,1063
4,Alabama,4br,1178,1178,1179,1180,1182,1184,1188.0,1191.0,...,1222,1227,1231,1236,1239,1239,1238,1236,1237,1236


Use pd.wide_to_long to transform month info to long

In [6]:
rental_copy = pd.wide_to_long(rental_copy, 
                              stubnames=['Price_2014', 'Price_2015', 'Price_2016', 'Price_2017'],
                              i=['Location', 'Bedroom_Size'],
                              j='month', 
                              sep='_').reset_index()

In [7]:
rental_copy.head()

,Location,Bedroom_Size,month,Price_2014,Price_2015,Price_2016,Price_2017
0,Alabama,Studio,01,573.0,580.0,580.0,592
1,Alabama,Studio,02,573.0,580.0,578.0,594
2,Alabama,Studio,03,574.0,580.0,576.0,595
3,Alabama,Studio,04,574.0,581.0,580.0,597
4,Alabama,Studio,05,575.0,582.0,583.0,599


Use pd.wide_to_long to transform year info to long

In [8]:
rental_copy = pd.wide_to_long(rental_copy, stubnames='Price', i=['Location', 'Bedroom_Size', 'month'], j='year', sep='_')\
    .reset_index()

In [9]:
rental_copy.head()

,Location,Bedroom_Size,month,year,Price
0,Alabama,Studio,01,2014,573.0
1,Alabama,Studio,01,2015,580.0
2,Alabama,Studio,01,2016,580.0
3,Alabama,Studio,01,2017,592.0
4,Alabama,Studio,02,2014,573.0


Change column headers to lower case

In [10]:
cols = ['location', 'bedroom_size', 'month', 'year', 'price']
rental_copy.columns = cols

In [11]:
rental_copy.columns

Index(['location', 'bedroom_size', 'month', 'year', 'price'], dtype='object')

Check data type of new columns

In [12]:
rental_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12240 entries, 0 to 12239
Data columns (total 5 columns):
location        12240 non-null object
bedroom_size    12240 non-null object
month           12240 non-null object
year            12240 non-null object
price           12155 non-null float64
dtypes: float64(1), object(4)
memory usage: 478.2+ KB


Because month and year won't be used for calculations, ok with leaving them as is.

In [13]:
rental_copy.to_csv('datasets/rent_by_type_1417.csv', index=False)